I haven't done this in a while. It used to be Yahoo Finance API, then Quantopian and Quandl, I'm way behind. I hear good things about (AI take the wheel) _Alpha Vantage_.

In [5]:
# %pip install joblib

In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from functools import lru_cache

load_dotenv()

@lru_cache(maxsize=None)
def fetch_dailies(ticker):
  url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&symbol={ticker}&apikey={os.getenv('ALPHAVANTAGE_API_KEY')}"
  response = requests.get(url)
  response = response.json()
  # Convert the response to a pandas DataFrame
  df = pd.DataFrame(response['Time Series (Daily)']).T
  df.index = pd.to_datetime(df.index)
  df.columns = ['Open', 'High', 'Low', 'Close', 'Adjusted_Close', 'Volume', 'Dividend_Amount', 'Split_Coefficient']
  df['Ticker'] = ticker
  # Convert 'Open' and 'Close' columns to numeric
  df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
  df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
  df['Adjusted_Close'] = pd.to_numeric(df['Adjusted_Close'], errors='coerce')
  df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
  df['Dividend_Amount'] = pd.to_numeric(df['Dividend_Amount'], errors='coerce')
  df['Split_Coefficient'] = pd.to_numeric(df['Split_Coefficient'], errors='coerce')
  
  # Calculate the overnight return
  df['Overnight_Return'] = (df['Open'] / df['Close'].shift(-1)) - 1
  # drop the last row, since we don't have the next day's data
  df = df.iloc[:-1]

  return df


In [3]:
fetch_dailies('AAPL')

,Open,High,Low,Close,Adjusted_Close,Volume,Dividend_Amount,Split_Coefficient,Ticker,Overnight_Return
2024-02-29,181.27,182.57,179.53,180.75,180.750000,136682597,0.0,1.0,AAPL,-0.000827
2024-02-28,182.51,183.12,180.13,181.42,181.420000,48953939,0.0,1.0,AAPL,-0.000657
2024-02-27,181.10,183.9225,179.56,182.63,182.630000,54318851,0.0,1.0,AAPL,-0.000331
2024-02-26,182.24,182.76,180.65,181.16,181.160000,40867421,0.0,1.0,AAPL,-0.001534
2024-02-23,185.01,185.04,182.23,182.52,182.520000,45119677,0.0,1.0,AAPL,0.003471
...,...,...,...,...,...,...,...,...,...,...
1999-11-08,87.75,97.73,86.75,96.37,0.728388,8490400,0.0,1.0,AAPL,-0.006341
1999-11-05,84.62,88.37,84.0,88.31,0.667468,3721500,0.0,1.0,AAPL,0.011959
1999-11-04,82.06,85.37,80.62,83.62,0.632020,3384700,0.0,1.0,AAPL,0.006871
1999-11-03,81.62,83.25,81.0,81.50,0.615997,2932700,0.0,1.0,AAPL,0.017072


Easy, awesome, I'm game.